# Fine-Tuning GPT-2 for Short Story Generation

This project demonstrates fine-tuning **GPT-2**, a pre-trained language model, to generate short stories using the **TinyStories** dataset. We'll preprocess the data, fine-tune the model with early stopping, evaluate its performance, and generate new stories.

In [1]:
from datasets import load_dataset

# Load the TinyStories dataset
dataset = load_dataset("roneneldan/TinyStories")

# Print dataset information
print(dataset)

c:\Users\jaisi\anaconda3\envs\transformers\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})


To reduce training time, we'll sample only **5% of the dataset** from both training and validation splits. This ensures faster experimentation while retaining representative data.

In [2]:
# Function to sample 5% of the dataset
def sample_five_percent(dataset_split):
    total_size = len(dataset_split)
    five_percent_size = total_size // 20  # 5% of the dataset
    return dataset_split.shuffle(seed=42).select(range(five_percent_size))

# Sample 5% from train and validation splits
train_data = sample_five_percent(dataset['train'])
val_data = sample_five_percent(dataset['validation'])

# Check sizes of sampled datasets
print(f"Train size (5%): {len(train_data)}")
print(f"Validation size (5%): {len(val_data)}")

Train size (5%): 105985
Validation size (5%): 1099


The **GPT-2 tokenizer** is used to preprocess text data by converting it into numerical format. We'll also ensure that the **end-of-sequence (EOS)** token is used for padding.

In [3]:
from transformers import GPT2Tokenizer

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set padding token to EOS token to avoid warnings
tokenizer.pad_token = tokenizer.eos_token

The dataset is tokenized into sequences of numerical tokens with **padding** and **truncation** applied to ensure all sequences are of the same length. This is critical for training in batches.

In [4]:
# Function to tokenize and prepare inputs/labels
def tokenize_function_with_labels(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding="max_length",  # Pad to a fixed length for batches
        max_length=512         # Set maximum length for sequences
    )
    # Add labels (same as input_ids for language modeling)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokenize train and validation datasets
train_dataset = train_data.map(tokenize_function_with_labels, batched=True, remove_columns=["text"])
val_dataset = val_data.map(tokenize_function_with_labels, batched=True, remove_columns=["text"])

# Verify tokenized data structure
print(train_dataset[0])

{'input_ids': [14967, 290, 32189, 588, 284, 711, 287, 262, 3952, 13, 1119, 766, 257, 1263, 3430, 319, 262, 2323, 13, 632, 318, 7586, 290, 890, 290, 4334, 13, 198, 198, 1, 8567, 11, 257, 3430, 2474, 5045, 1139, 13, 366, 40, 460, 10303, 340, 2474, 198, 198, 1544, 8404, 284, 10303, 262, 3430, 11, 475, 340, 318, 1165, 5802, 13, 679, 8953, 866, 290, 10532, 262, 3430, 13, 198, 198, 1, 46, 794, 2474, 339, 1139, 13, 366, 2504, 5938, 2474, 198, 198, 44, 544, 22051, 13, 1375, 318, 407, 1612, 11, 673, 655, 6834, 340, 318, 8258, 13, 198, 198, 1, 5756, 502, 1949, 2474, 673, 1139, 13, 366, 40, 460, 5236, 340, 2474, 198, 198, 3347, 11103, 510, 262, 3430, 290, 7584, 340, 319, 607, 1182, 13, 1375, 11114, 6364, 290, 7773, 13, 1375, 857, 407, 2121, 866, 13, 198, 198, 1, 22017, 2474, 5045, 1139, 13, 366, 1639, 389, 922, 379, 22486, 2474, 198, 198, 1, 10449, 345, 2474, 32189, 1139, 13, 366, 1026, 318, 1257, 2474, 198, 198, 2990, 1011, 4962, 22486, 262, 3430, 319, 511, 6665, 11, 5101, 11, 290, 7405, 13, 111

We load the pre-trained **GPT-2** model and adjust the token embeddings to account for the padding token added during tokenization.

In [5]:
from transformers import GPT2LMHeadModel

# Load GPT-2 model with a language modeling head
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Resize embeddings to account for padding token
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

Training arguments control various aspects of training, including **learning rate**, **batch size**, **evaluation frequency**, and saving model checkpoints. The best model will be loaded at the end of training.

In [6]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-tinystories",  # Directory to save model checkpoints
    evaluation_strategy="steps",     # Evaluate at specific intervals
    eval_steps=500,                  # Evaluate every 500 steps
    learning_rate=5e-5,              # Learning rate for optimizer
    weight_decay=0.01,               # Regularization strength
    per_device_train_batch_size=4,   # Training batch size
    per_device_eval_batch_size=4,    # Evaluation batch size
    num_train_epochs=3,              # Total number of training epochs
    save_strategy="steps",           # Save model at specific intervals
    save_steps=500,                  # Save model every 500 steps
    logging_dir="./logs",            # Directory for training logs
    save_total_limit=2,              # Limit total number of saved checkpoints
    load_best_model_at_end=True,     # Load best model after training
)

c:\Users\jaisi\anaconda3\envs\transformers\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


An **early stopping callback** halts training if the validation loss doesn't improve for 2 consecutive evaluation steps, preventing unnecessary computation.

In [7]:
from transformers import EarlyStoppingCallback

# Add EarlyStoppingCallback to stop training if validation loss does not improve
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)

The **Trainer** API simplifies training by integrating the model, dataset, training arguments, and callbacks into one interface.

In [8]:
from transformers import Trainer

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback],  # Add early stopping
)

C:\Users\jaisi\AppData\Local\Temp\ipykernel_3740\3006593948.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


The model is fine-tuned on the **TinyStories** dataset, with evaluation and checkpointing performed every 500 steps. Early stopping ensures training efficiency.

In [9]:
# Train the model
trainer.train()

  1%|          | 500/79491 [03:02<7:56:07,  2.77it/s]

{'loss': 0.8855, 'grad_norm': 1.7314858436584473, 'learning_rate': 4.968549898730674e-05, 'epoch': 0.02}


                                                     
  1%|          | 500/79491 [03:30<7:56:07,  2.77it/s]

{'eval_loss': 0.770193874835968, 'eval_runtime': 27.9024, 'eval_samples_per_second': 39.387, 'eval_steps_per_second': 9.856, 'epoch': 0.02}


  1%|▏         | 1000/79491 [06:33<8:07:16,  2.68it/s] 

...

                                                       
 99%|█████████▉| 79000/79491 [9:18:02<02:57,  2.77it/s]

{'eval_loss': 0.5740256905555725, 'eval_runtime': 27.2687, 'eval_samples_per_second': 40.303, 'eval_steps_per_second': 10.085, 'epoch': 2.98}


100%|██████████| 79491/79491 [9:21:03<00:00,  2.36it/s]

{'train_runtime': 33663.1059, 'train_samples_per_second': 9.445, 'train_steps_per_second': 2.361, 'train_loss': 0.6290795461238095, 'epoch': 3.0}


TrainOutput(global_step=79491, training_loss=0.6290795461238095, metrics={'train_runtime': 33663.1059, 'train_samples_per_second': 9.445, 'train_steps_per_second': 2.361, 'total_flos': 8.307910803456e+16, 'train_loss': 0.6290795461238095, 'epoch': 3.0})

After training, we evaluate the model on the validation set to measure its performance.

In [10]:
# Evaluate the model on the validation set
results = trainer.evaluate()
print("Evaluation Results:", results)

100%|██████████| 275/275 [00:27<00:00,  9.96it/s]

Evaluation Results: {'eval_loss': 0.5740256905555725, 'eval_runtime': 27.6359, 'eval_samples_per_second': 39.767, 'eval_steps_per_second': 9.951, 'epoch': 3.0}


Finally, we use the fine-tuned model to generate a short story based on a given prompt. The model completes the story in a coherent and creative manner.

In [11]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "Once upon a time in a magical forest"
generated_story = story_generator(prompt, max_length=150, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Story:
Once upon a time in a magical forest, there was a bunny named Benny. Benny loved to hop and play all day in his green grass. One day, Benny noticed a big tree with lots of leaves. He hopped over to it and saw something very yummy! Benny wanted to eat the taste, so he started eating the yummy thing.

Suddenly, a big bird flew down and grabbed Benny. It tried to take Benny away, but Benny didn't want to give up. He hopped back to his burrow and watched the little bird eat the yummy thing. Benny couldn't believe his eyes!

Later that day, Benny realized that he should be more careful around the tree. He hopped around it a little bit and
